# 通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集

In [1]:
import numpy as np
import  torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据,features和labels作为API的参数传递，并通过数据迭代器指定batch_size,布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个PyTorch数据迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[ 0.5533, -1.3179],
         [-1.4095, -1.9095],
         [ 0.5070,  0.9571],
         [ 1.4586, -1.3796],
         [ 1.4516, -0.4831],
         [ 0.2781, -0.1334],
         [ 0.0961,  0.5555],
         [-0.1886, -1.2585],
         [ 0.4601,  0.0167],
         [-2.3172, -1.5602]]),
 tensor([[ 9.8040],
         [ 7.8721],
         [ 1.9456],
         [11.7989],
         [ 8.7279],
         [ 5.2076],
         [ 2.5078],
         [ 8.1014],
         [ 5.0627],
         [ 4.8670]])]

使用框架的预定义好的层

In [8]:
# nn是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2,1)) # 输入2 输出1

初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

定义损失函数:计算均方误差使用的是MSELoss类，也称为平方$L_2$范数,默认情况下，它返回所有样本损失的平均值

In [10]:
loss = nn.MSELoss()

定义优化算法实例化SGD,只需要设置lr值，这里设置为0.03

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似,在每个迭代周期里，我们将完整遍历一次数据集（train_data），不停地从中获取一个小批量的输入和相应的标签。对于每一个小批量，我们会进行以下步骤:
通过调用net(X)生成预测并计算损失l（前向传播）。
通过进行反向传播来计算梯度。
通过调用优化器来更新模型参数。
为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程

In [13]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000101
epoch 2, loss 0.000102
epoch 3, loss 0.000101


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [14]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-4.1986e-04,  2.6941e-05])
b的估计误差： tensor([-0.0003])
